In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
num_epochs = 4
batch_size = 4
learning_rate= 0.01

In [5]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [6]:
test_dataset = torchvision.datasets.CIFAR10(root="/data",
                                            train=True,
                                            download=True,
                                            transform=transform)
train_dataset = torchvision.datasets.CIFAR10(root="/data",
                                            train=False,
                                            download=True,
                                            transform=transform)

100%|██████████| 170M/170M [00:04<00:00, 35.6MB/s]


In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [8]:
classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

In [18]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5) #3rgb, outputchannelsize, 5x5kernelsize (output = (32 - 5 / 1) + 1 = 28 = 6x28x28)
    self.pool = nn.MaxPool2d(2, 2)# kernel size = 2, stride=2 (output = (28 - 2 / 2) + 1 = 14 = 6x14x14)
    self.conv2 = nn.Conv2d(6, 16, 5) # (output = (14 - 5 / 1) + 1 = 10 = 16x10x10)
    self.fc1 = nn.Linear(16*5*5, 120) # after one more pooling layer the size is (16x5x5)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [19]:
model = ConvNet().to(device)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [21]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 2000 == 0:
      print(f"Epoch {epoch+1} step {i+1} loss {loss.item():.4f}")

Epoch 1 step 2000 loss 2.0007
Epoch 2 step 2000 loss 1.4474
Epoch 3 step 2000 loss 1.1328
Epoch 4 step 2000 loss 1.1382


In [23]:

# Set the model to evaluation mode
model.eval()

y_true = []
y_pred = []

with torch.no_grad():  # disable gradient calculation (faster + less memory)
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)   # get class index with highest score

        # Save true and predicted labels
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# ✅ Classification report
print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=[
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]))


Classification Report:

              precision    recall  f1-score   support

    airplane       0.43      0.66      0.52      5000
  automobile       0.52      0.69      0.59      5000
        bird       0.34      0.43      0.38      5000
         cat       0.31      0.35      0.33      5000
        deer       0.39      0.29      0.34      5000
         dog       0.40      0.45      0.42      5000
        frog       0.59      0.44      0.50      5000
       horse       0.59      0.50      0.54      5000
        ship       0.63      0.43      0.51      5000
       truck       0.60      0.35      0.45      5000

    accuracy                           0.46     50000
   macro avg       0.48      0.46      0.46     50000
weighted avg       0.48      0.46      0.46     50000

